In [22]:
import pandas as pd 
import numpy as np 
import time
import tqdm
import re

import bs4 
import glob
import json

pd.set_option("max.columns", 131)

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Read the data

In [2]:
df = pd.read_csv('artigos_medium_classificados.csv', sep = ',', dtype = {'author_url':str}).dropna(subset = ['classificacao', 'author_url'])
df.shape

(1155, 8)

In [4]:
df.classificacao.sum()

155.0

# Pre-processing

In [3]:
df['date'] = pd.to_datetime(df.date, format = '%d/%m/%Y')

In [5]:
df.claps = df.claps.str.replace(r'[Kk]', 'e3',regex = True,).astype(float)

In [6]:
df.title = df.title.str.replace(r'[\xa0]', '',regex = True)

In [7]:
df.title = df.title.str.replace(r'\W', ' ', regex = True)

In [8]:
df.title[0]

'Implementing an Autoencoder in PyTorch'

In [9]:
df['author_name'] = df.author_url.apply(lambda x: x.split('/@')[1])

In [10]:
features = df.drop(columns = ['classificacao', 'author_url'])
y = df['classificacao'].copy()


def tokenize(text):
    text = re.sub(r"[^a-zA-z0-9]"," ", unidecode(text))
    clean_tokens = []
    
    tokens = word_tokenize(text, language='portuguese')
    tokens = [w for w in tokens if w not in stopwords.words("portuguese")]
    lemmatizer = RSLPStemmer()
        
    for tok in tokens:
        clean_tok = lemmatizer.stem(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.5, random_state=42)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = X_train['title']
title_test = X_test['title']

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_test)

In [43]:
from scipy.sparse import hstack, vstack

X_train_title = hstack([X_train[['claps', 'responses']], title_bow_train])
X_test_title = hstack([X_test[['claps', 'responses']], title_bow_val])
X_train_title.shape, X_test_title.shape, y_train.shape, y_test.shape

((577, 812), (578, 812), (577,), (578,))

## Random Forest

In [44]:
mdl_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl_rf.fit(X_train_title, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [45]:
p_rf = mdl_rf.predict_proba(X_test_title)[:, 1]

In [46]:
average_precision_score(y_test, p_rf), roc_auc_score(y_test, p_rf)

(0.24966831399742445, 0.6851803313063536)

## LGBM

In [34]:
params = [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 3]
lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]

min_df = params[6]
ngram_range = (1, params[7])


In [36]:
title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_test)


In [38]:
title_bow_train

<577x810 sparse matrix of type '<class 'numpy.float64'>'
	with 4031 stored elements in Compressed Sparse Row format>

In [51]:
X_train_title = hstack([X_train[['claps', 'responses']], title_bow_train])
X_test_title = hstack([X_test[['claps', 'responses']], title_bow_val])
X_train_title.shape, X_test_title.shape, y_train.shape, y_test.shape

((577, 812), (578, 812), (577,), (578,))

In [48]:

mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight="balanced", n_jobs=6)
mdl_lgbm.fit(X_train_title, y_train)

p_lgbm = mdl_lgbm.predict_proba(X_test_title)[:, 1]

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [49]:
average_precision_score(y_test, p_lgbm), roc_auc_score(y_test, p_lgbm)

(0.19102626506415354, 0.5968756552736424)

## Logistic regression

In [52]:
from sklearn.pipeline import make_pipeline

In [55]:
Xtrain_wtitle2 = csr_matrix(X_train_title.copy())
Xval_wtitle2 = csr_matrix(X_test_title.copy())

#scaler = StandardScaler()
#scaler = MaxAbsScaler()

#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.5, penalty='l2',n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, y_train)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [ ]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

In [ ]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

## Ensemble

In [ ]:
p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

In [ ]:
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgbm}).corr()

In [ ]:
p = 0.5*p_rf + 0.5*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p)

## Salvar modelos

In [53]:
import joblib as jb

In [ ]:
jb.dump(mdl_lgbm, "lgbm_20200208.pkl.z")
jb.dump(mdl_rf, "random_forest_20200208.pkl.z")
#jb.dump(lr_pipeline, "logistic_reg_20200208.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200208.pkl.z")